In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from PIL import Image 
import numpy as np
import re
import os 
import glob 
import random
import logging
from tensorflow.keras import backend as K
import math
import cv2
tf.get_logger().setLevel(logging.ERROR)

In [60]:
def extract_channels(bayer):
    r, g, b = tf.zeros_like(bayer), tf.zeros_like(bayer), tf.zeros_like(bayer)
    r[::2, ::2] = bayer[::2, ::2]  # Red channel
    g[1::2, ::2] = bayer[1::2, ::2]  # Green channel for even rows
    g[::2, 1::2] = bayer[::2, 1::2]  # Green channel for odd rows
    g[1::2, 1::2] = bayer[1::2, 1::2]  # Blue channel
    return r, g, b

In [ ]:
# red channel 
    pad_r = np.pad(r, (1,1), mode='constant')
    pad_r[1:-1:2, 2:-1:2] = (pad_r[1:-1:2, 1:-1:2] + pad_r[1:-1:2, 3::2]) / 2
    pad_r[2:-1:2, 1:-1:2] = (pad_r[1:-1:2, 1:-1:2] + pad_r[3::2, 1:-1:2]) / 2
    pad_r[2:-1:2, 2:-1:2] = (pad_r[1:-1:2, 1:-1:2] + pad_r[3::2, 1:-1:2] + pad_r[1:-1:2, 3::2] + pad_r[3::2, 3::2]) / 4
    new_r = pad_r[1:-1, 1:-1]
    # green channel 
    pad_g = np.pad(g, (1,1), mode='constant')
    pad_g[1:-1:2, 1:-1:2] = (pad_g[0:-2:2, 1:-1:2] + pad_g[1:-1:2, 0:-2:2] + pad_g[1:-1:2, 2::2] + pad_g[2::2, 1:-1:2]) / 4
    pad_g[2::2, 2::2] = (pad_g[2::2, 1:-2:2] + pad_g[1:-1:2, 2::2] + pad_g[2::2, 3::2] + pad_g[3::2, 2::2]) / 4
    new_g = pad_g[1:-1, 1:-1]
    # blue channel 
    pad_b = np.pad(b, (1,1), mode='constant')
    pad_b[1:-1:2, 2:-1:2] = (pad_b[0:-2:2, 2:-1:2] + pad_b[2::2, 2::2]) / 2
    pad_b[2:-1:2, 1:-1:2] = (pad_b[2::2, 0:-2:2] + pad_b[2::2, 2::2]) / 2
    pad_b[1:-1:2, 1:-1:2] = (pad_b[0:-2:2, 0:-2:2] + pad_b[0:-2:2, 2:-1:2] + pad_b[2:-1:2, 0:-2:2] + pad_b[2::2, 2::2]) / 4
    new_b = pad_b[1:-1, 1:-1]

In [ ]:
"""
workable tf ops
tf.concact([r,g,b], axis=-1) work like stack 
tf.zeros(shape)
"""


In [257]:
class Debayer(layers.Layer):
    def __init__(self):
        super(Debayer, self).__init__()
        
    def call(self, inputs):
        
        return rgb

In [258]:
def build_net(input_shape=(324,324,1)):
    # (None, 324, 324, 1)
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = Debayer()(inputs)
    model = Model(inputs, x, name="demosaic")
    return model

In [259]:
def representative_dataset():
    dataset_dir = "/app/event_detection/data/train/something"
    files = glob.glob(dataset_dir + "/*.bayer")
    for file in files:
        img = tf.io.read_file(file)
        img = tf.io.decode_raw(img, tf.uint8)
        img = tf.cast(img, tf.float32) / 255.
        img_shape = [324, 324, 1]
        img = tf.reshape(img, img_shape)
        img = tf.expand_dims(img, axis=0)
        yield [img]

In [260]:
model = build_net()

(1, 324, 324, 1)
(None, 162, 162, 1)
(1, 162, 162, 1)


TypeError: in user code:

    <ipython-input-257-530341d656c7>:12 call  *
        r_np[..., ::2, ::2, :] = inputs[..., ::2, ::2, :]

    TypeError: __array__() takes 1 positional argument but 2 were given


In [237]:
rggb_tile = np.array([[1, 2], [3, 4]], dtype=np.float32)
repeats = (162, 162)
rggb_tiles = np.tile(rggb_tile, repeats)
inputs = tf.cast(rggb_tiles, tf.float32)
inputs = tf.expand_dims(tf.expand_dims(inputs, axis=-1), axis=0)
output = model(inputs)
print(output.shape)
# print(np.squeeze(output.numpy()))

(1, 324, 324)


In [238]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

save_path = "/app/event_detection/model.tflite"

with open(save_path, 'wb') as f:
    f.write(tflite_model)

ConverterError: /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1024:0: error: 'tf.StridedSlice' op is neither a custom op nor a flex op
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/dispatch.py:201:0: note: called from
<ipython-input-233-3b7c4a90aa88>:7:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:302:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:985:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:508:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:386:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:985:0: note: called from
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/saving/saving_utils.py:134:0: note: called from
/usr/local/lib/python3.6/dist-p

In [214]:
!edgetpu_compiler model.tflite

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 17 ms.

Input model: model.tflite
Input size: 10.95MiB
Output model: model_edgetpu.tflite
Output size: 168.00B
On-chip memory used for caching model parameters: 0.00B
On-chip memory remaining for caching model parameters: 0.00B
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 0
Total number of operations: 0
Operation log: model_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 


# Inference

In [1]:
from pycoral.utils import edgetpu
from pycoral.adapters import common
from PIL import Image
import numpy as np
import time
import os 
import re
import glob 
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt

def visualize(original, augmented, figure_size=(10,5)):
    fig = plt.figure(figsize=figure_size)
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)
    plt.axis("off")

In [ ]:
def prepare_input_bayer(bayer_path, img_w=324, img_h=324):
    bayer = tf.io.read_file(bayer_path)
    bayer = tf.io.decode_raw(bayer, tf.uint8)
    bayer = tf.cast(bayer, tf.float32) / 255.
    bayer_shape = [324, 324, 1]
    bayer = tf.reshape(bayer, bayer_shape)
    bayer = tf.expand_dims(bayer, axis=0)
    return bayer

In [ ]:
def quantized_input(input_tensor, input_scale, input_zero_point):
    return tf.cast((input_tensor / input_scale + input_zero_point), tf.uint8)

In [ ]:
def dequantized_output(output_tensor, output_scale, output_zero_point):
    return tf.cast(((output_tensor - output_zero_point) * output_scale), tf.float32)

In [19]:
model_path = "/app/event_detection/model_edgetpu.tflite"
interpreter = edgetpu.make_interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

In [46]:
bayer_path = "/app/event_detection/data/train/something/1690946037374_3689508_0_3e08da56-2b83-f591-35f5-10e7c21b3d5f.bayer"
bayer = prepare_input_bayer(bayer_path)
print(bayer.shape)

(1, 324, 324, 1)


In [49]:
input_data = quantized_input(bayer, input_scale, input_zero_point)
print(input_data.shape)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
deq_output = dequantized_output(output_data, output_scale, output_zero_point)

(1, 324, 324, 1)


In [51]:
deq_output.shape

TensorShape([1, 324, 324, 8])